In [1]:

import cv2
import time
import numpy as np

# -----------------------------
# Step 1: Setup Video Writer
# -----------------------------
video_format = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('cloak_output.avi', video_format, 20.0, (640, 480))

# -----------------------------
# Step 2: Start Webcam
# -----------------------------
camera = cv2.VideoCapture(0)

# Let camera warm up
time.sleep(3)

frame_count = 0
bg_frame = None

# -----------------------------
# Step 3: Capture Background
# -----------------------------
for i in range(60):
    success, bg_frame = camera.read()
bg_frame = np.flip(bg_frame, axis=1)

# -----------------------------
# Step 4: Process Live Frames
# -----------------------------
while camera.isOpened():
    ret, current_frame = camera.read()
    if not ret:
        break
    
    frame_count += 1
    current_frame = np.flip(current_frame, axis=1)
    
    # Convert to HSV color space
    hsv_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2HSV)
    
    # Detect cloak color (Red in this case)
    red_lower1 = np.array([0, 120, 50])
    red_upper1 = np.array([10, 255, 255])
    mask_red1 = cv2.inRange(hsv_frame, red_lower1, red_upper1)
    
    red_lower2 = np.array([170, 120, 70])
    red_upper2 = np.array([180, 255, 255])
    mask_red2 = cv2.inRange(hsv_frame, red_lower2, red_upper2)
    
    cloak_mask = mask_red1 + mask_red2
    
    # Refine the mask
    cloak_mask = cv2.morphologyEx(cloak_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    cloak_mask = cv2.morphologyEx(cloak_mask, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8))
    
    # Inverted mask
    inverse_mask = cv2.bitwise_not(cloak_mask)
    
    # Segment current frame
    visible_part = cv2.bitwise_and(current_frame, current_frame, mask=inverse_mask)
    
    # Segment background
    invisible_part = cv2.bitwise_and(bg_frame, bg_frame, mask=cloak_mask)
    
    # Combine both
    final_frame = cv2.addWeighted(visible_part, 1, invisible_part, 1, 0)
    
    # Write & show output
    output_video.write(final_frame)
    cv2.imshow("Invisible Cloak Effect", final_frame)
    
    if cv2.waitKey(1) == 27:  # press 'Esc' to exit
        break

# -----------------------------
# Step 5: Release Resources
# -----------------------------
camera.release()
output_video.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 